# Data Preprocessing

In [1]:
# import packages
import numpy as np
import pandas as pd
import re

As a review from the last notebook, we scraped speech transcripts from James Clear's website on his ["Great Speeches" page](https://jamesclear.com/great-speeches) (where we took the top 10 which are sorted alphabetically by the orator's last name) and the lyrics from all songs in bestselling albums of all time (according to [Business Insider](https://www.businessinsider.com/50-best-selling-albums-all-time-2016-9)) from [AZLyrics](https://www.azlyrics.com/). 

Below are what our raw dataframes look like. Not bad if I do say so myself. Of course, there's a lot of preprocessing that has to be done, but it looks like we properly collected the data.

In [2]:
speech_df = pd.read_excel('speeches.xlsx', dtype=str)
speech_df

,Orator,Title,Transcript,Link
0,Chimamanda Ngozi Adichie,The Danger of a Single Story,I'm a storyteller. And I would like to tell yo...,https://jamesclear.com/great-speeches/the-dang...
1,Jeff Bezos,What Matters More Than Your Talents,"As a kid, I spent my summers with my grandpare...",https://jamesclear.com/great-speeches/what-mat...
2,John C. Bogle,Enough,Here’s how I recall the wonderful story that s...,https://jamesclear.com/great-speeches/enough-b...
3,Brené Brown,The Anatomy of Trust,"Oh, it just feels like an incredible understat...",https://jamesclear.com/great-speeches/the-anat...
4,John Cleese,Creativity in Management,"You know, when Video Arts asked me if I'd like...",https://jamesclear.com/great-speeches/creativi...
5,William Deresiewicz,Solitude and Leadership,My title must seem like a contradiction. What ...,https://jamesclear.com/great-speeches/solitude...
6,Richard Feynman,Seeking New Laws,What I want to talk to you about tonight is st...,https://jamesclear.com/great-speeches/seeking-...
7,Neil Gaiman,Make Good Art,I never really expected to find myself giving ...,https://jamesclear.com/great-speeches/make-goo...
8,John W. Gardner,Personal Renewal,I'm going to talk about “Self-Renewal.” One of...,https://jamesclear.com/great-speeches/personal...
9,Elizabeth Gilbert,Your Elusive Creative Genius,I am a writer. Writing books is my profession ...,https://jamesclear.com/great-speeches/your-elu...


In [3]:
song_df =  pd.read_excel('songs.xlsx', dtype=str)
song_df

,Artist,Album,Lyric,Artist_Link
0,Michael Jackson,"""Thriller""",\n\r\nI said you wanna be startin' somethin'\n...,https://www.azlyrics.com/j/jackson.html
1,Eagles,"""Hotel California""","\n\r\nOn a dark desert highway, cool wind in m...",https://www.azlyrics.com/e/eagles.html
2,Led Zeppelin,"""Led Zeppelin IV""","\n\r\nHey, hey mama, said the way you move\nGo...",https://www.azlyrics.com/l/ledzeppelin.html
3,Pink Floyd,"""The Wall""",\n\r\n(We came in)\n\nSo ya thought ya might l...,https://www.azlyrics.com/p/pinkfloyd.html
4,AC/DC,"""Back In Black""",\n\r\nI'm rolling thunder pouring rain\nI'm co...,https://www.azlyrics.com/a/acdc.html
5,Garth Brooks,"""Double Live""",\n\r\nWe all came here for a party tonight\nAn...,https://www.azlyrics.com/b/brooks.html
6,Hootie & The Blowfish,"""Cracked Rear View""",\n\r\nYou got your big girl \nNow you've got y...,https://www.azlyrics.com/h/hootie.html
7,Fleetwood Mac,"""Rumours""",\n\r\nI know there's nothing to say\nSomeone h...,https://www.azlyrics.com/f/fleetwood.html
8,Shania Twain,"""Come On Over""",\n\r\nLet's go girls! Come on.\n\nI'm going ou...,https://www.azlyrics.com/t/twain.html
9,The Beatles,"""The Beatles (The White Album)""","\n\r\nOh, flew in by Miami Beach B.O.A.C.\nDid...",https://www.azlyrics.com/b/beatles.html


Below, I've combined our two raw data frames in the hopes that they can be preprocessed at the same time. Twenty rows may be a bit much for some people, but it's simple to take out rows so I felt it was best to have more rather than less. As an aside it also made webscraping a more interesting exercise since I needed to deal with different page and url formats.

In [4]:
comm_df =  pd.read_excel('communication_data.xlsx', dtype=str)
comm_df

,Originator,Title,Corpus,Source
0,Chimamanda Ngozi Adichie,The Danger of a Single Story,I'm a storyteller. And I would like to tell yo...,https://jamesclear.com/great-speeches/the-dang...
1,Jeff Bezos,What Matters More Than Your Talents,"As a kid, I spent my summers with my grandpare...",https://jamesclear.com/great-speeches/what-mat...
2,John C. Bogle,Enough,Here’s how I recall the wonderful story that s...,https://jamesclear.com/great-speeches/enough-b...
3,Brené Brown,The Anatomy of Trust,"Oh, it just feels like an incredible understat...",https://jamesclear.com/great-speeches/the-anat...
4,John Cleese,Creativity in Management,"You know, when Video Arts asked me if I'd like...",https://jamesclear.com/great-speeches/creativi...
5,William Deresiewicz,Solitude and Leadership,My title must seem like a contradiction. What ...,https://jamesclear.com/great-speeches/solitude...
6,Richard Feynman,Seeking New Laws,What I want to talk to you about tonight is st...,https://jamesclear.com/great-speeches/seeking-...
7,Neil Gaiman,Make Good Art,I never really expected to find myself giving ...,https://jamesclear.com/great-speeches/make-goo...
8,John W. Gardner,Personal Renewal,I'm going to talk about “Self-Renewal.” One of...,https://jamesclear.com/great-speeches/personal...
9,Elizabeth Gilbert,Your Elusive Creative Genius,I am a writer. Writing books is my profession ...,https://jamesclear.com/great-speeches/your-elu...


In [5]:
chars_to_keep = r"' "
def is_desired_char(c):
    return (c.isalpha() or c in chars_to_keep)

def keep_words(s):
    ret_str = ''.join(c for c in s if is_desired_char(c))
    return ' '.join(ret_str.split(' '))



In [6]:
# speech_df['Corpus'] = speech_df['Corpus'].str.lower()
# speech_df['Corpus'] = speech_df['Corpus'].apply(keep_words)

## Data Cleaning

### Topics
* tf-idf
* stemming

Not going to remove #s